### Example of displaying heatmap using folium

#### Import libraries  

In [ ]:
import polars as pl
import folium

#### Parameters

In [ ]:
csv_in = 'pop_move_H27_H28-utf8.csv'

pl.Config().set_tbl_cols(999).set_tbl_rows(999)

#### Read CSV file

In [ ]:
df = pl.read_csv(csv_in, separator=',')
print(df.head())

#### Calculation of increase/decrease of population  

In [ ]:
df = df.with_columns(
    (pl.col('転入') - pl.col('転出')).alias('増減')
)
display(df.head())

#### Drop lines other than Prefs  

In [ ]:
df['都道府県'].str.contains(r'^(.*市|.*計|.*区部)$').count()  # debug

In [ ]:
df2 = df.filter(~pl.col("都道府県").str.contains(r'^(.*市|.*計|.*区部)$'))
display(df2.describe())

#### Check data  

#### Parameters2 (for lat/lon)   

In [ ]:
csv_in2 = 'h3010pref_pos_utf8.csv'

#### Read CSV file

In [ ]:
df_pos = pl.read_csv(csv_in2)
print(df_pos.shape)
display(df_pos.head())

#### Merge DataFrames to add lat/lon to pref data  

In [ ]:
df_merged = df2.join(df_pos, left_on='都道府県', right_on='name', how='left')
print(df_merged.shape)
display(df_merged.head())

#### Make data for heatmap  
[ [lat2, lon2, value2], [lat2, lon2, value2], ... ]  

In [ ]:
df_heat = df_merged[['lat', 'lon', '増減']].to_pandas()
print(df_heat.shape)
display(df_heat.head())

#### Make map  

In [ ]:
from folium.plugins import HeatMap

m = folium.Map(location=[35, 140], zoom_start=4)
m.add_child(HeatMap(df_heat, radius=12))
display(m)

#### Add color bar  

In [ ]:
### Add color bar

from folium.plugins import HeatMap
import branca.colormap as cm
import numpy as np

m = folium.Map(location=[35, 140], zoom_start=4)
m.add_child(HeatMap(df_heat, radius=12,gradient={"0.4":"blue", "0.6":"cyan", "0.7":"lime", "0.8":"yellow", "1.0":"red"}))

min = df_heat['増減'].min()
max = df_heat['増減'].max()
colors = ['blue', 'cyan', 'lime', 'yellow', 'red']
index = np.array([.4, .6, .7, .8, 1.])*(max-min)+min
colormap = cm.LinearColormap(colors, index=index, vmin=min, vmax=max).to_step(100)
colormap.caption = 'Increase/Decrease of Population'
m.add_child(colormap)

display(m)